In [1]:
# Imports

import requests
from bs4 import BeautifulSoup
import re
from playwright.async_api import async_playwright
import pandas as pd
import time
import json
import numpy as np
from pandas import json_normalize

In [2]:
# Shamelessly using mapbox tutorial code

with open('../map_files/US_JD_simply.json') as json_data:
    geometry_data = json.load(json_data)



In [3]:
# Reading a dataframe from it

districts_df = pd.DataFrame.from_dict(json_normalize(geometry_data['features']), orient='columns')

districts_df.head()

,type,id,geometry.type,geometry.coordinates,properties.ABBR,properties.DISTRICT,properties.SQ_Miles,properties.Shape_Leng,properties.Shape_Area,properties.District_N,properties.Shape__Are,properties.Shape__Len
0,Feature,1,Polygon,"[[[-85.30506134, 33.482673645], [-85.263984679...",ALM,ALABAMA MIDDLE,1.557104e+04,9.918902,3.851034,11,3.851034,9.918902
1,Feature,2,Polygon,"[[[-87.9870452879999, 35.0075187680001], [-87....",ALN,ALABAMA NORTHERN,2.279241e+04,12.526683,5.752874,11,5.752874,12.526683
2,Feature,3,MultiPolygon,"[[[[-88.1121139529999, 30.2584381100002], [-88...",ALS,ALABAMA SOUTHERN,3.672870e+04,14.291408,3.267182,11,3.267182,14.291408
3,Feature,4,MultiPolygon,"[[[[-179.098083496, 51.3043937680002], [-179.0...",AK,ALASKA,2.130451e+07,787.366223,279.696379,9,279.696379,787.366223
4,Feature,5,Polygon,"[[[-109.044883728, 36.9986305240002], [-109.04...",AZ,ARIZONA,1.140598e+05,23.752602,28.930993,9,28.930993,23.752602


In [4]:
# reading the orgs dataframe from csv.

orgs_df = pd.read_csv('../output/orgs.csv')


In [5]:
# The dataframe of organizations is way longer than the number of districts! Making a list of dictionaries with information to smush into a dataframe.

districts = []

districts_index = orgs_df.groupby('district').name.count().index

for district in districts_index:
    districts.append(district)

In [6]:

orgs = []

for district in districts:
    orgs.append(orgs_df[orgs_df.district == district].groupby('district').name.count().values[0])

In [7]:
top_judges = []


for district in districts:

      top_judges.append(orgs_df[orgs_df.district == district].groupby('district').name.value_counts().index[0][1])

In [8]:
most_orgs = []

for district in districts:
    most_orgs.append(orgs_df[orgs_df.district == district].groupby('district').name.value_counts().values[0])

In [9]:
big_list = []

for i in range(0, len(districts)):
    temp_dict = {
        'district': districts[i],
        'total_orgs': orgs[i],
        'top_judge': top_judges[i],
        'top_judge_orgs': most_orgs[i]
    }
    big_list.append(temp_dict)

big_list[0]


{'district': ' central district of california',
 'total_orgs': 17,
 'top_judge': 'Hernan D. Vera',
 'top_judge_orgs': 5}

In [10]:
df = pd.DataFrame(big_list)

In [11]:
# Formatting.

df.district = df.district.str.title().str.strip()
df.district = df.district.str.title().str.strip()
df.district = df.district.str.replace(' Of ', ' of ')

df.head()

,district,total_orgs,top_judge,top_judge_orgs
0,Central District of California,17,Hernan D. Vera,5
1,Central District of Illinois,6,Colleen R. Lawless,6
2,District of Colorado,37,Tiffany M. Cartwright,13
3,District of Columbia,37,Todd E. Edelman,17
4,District of Connecticut,12,Sarala Vidya Nagala,11


In [12]:
# I need a field to merge the dataframes on.

districts_df['match'] = districts_df['properties.DISTRICT'].str.lower()
df['match'] = df.district.str.lower().str.split('district of').str[-1] + " " + df.district.str.lower().str.split('district of').str[0]

df.match = df.match.str.strip()

In [13]:
# The only district (besides some territories that had no data anyway) that this doesn't work for is the District of Columbia, so I have to fix it manually.

for i in range(len(df.match)):
    if df.match[i] == 'columbia':
        df.match[i] = 'district of columbia'
df.match[0:5]

/var/folders/8w/c7bhb4y56693wcs88hywlmq00000gn/T/ipykernel_21352/273093132.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.match[i] = 'district of columbia'


0      california central
1        illinois central
2                colorado
3    district of columbia
4             connecticut
Name: match, dtype: object

In [14]:
# Merging!

merged_df = pd.merge(districts_df, df, left_on='match', right_on='match', how='left')
merged_df.head()

,type,id,geometry.type,geometry.coordinates,properties.ABBR,properties.DISTRICT,properties.SQ_Miles,properties.Shape_Leng,properties.Shape_Area,properties.District_N,properties.Shape__Are,properties.Shape__Len,match,district,total_orgs,top_judge,top_judge_orgs
0,Feature,1,Polygon,"[[[-85.30506134, 33.482673645], [-85.263984679...",ALM,ALABAMA MIDDLE,1.557104e+04,9.918902,3.851034,11,3.851034,9.918902,alabama middle,NaN,NaN,NaN,NaN
1,Feature,2,Polygon,"[[[-87.9870452879999, 35.0075187680001], [-87....",ALN,ALABAMA NORTHERN,2.279241e+04,12.526683,5.752874,11,5.752874,12.526683,alabama northern,NaN,NaN,NaN,NaN
2,Feature,3,MultiPolygon,"[[[[-88.1121139529999, 30.2584381100002], [-88...",ALS,ALABAMA SOUTHERN,3.672870e+04,14.291408,3.267182,11,3.267182,14.291408,alabama southern,NaN,NaN,NaN,NaN
3,Feature,4,MultiPolygon,"[[[[-179.098083496, 51.3043937680002], [-179.0...",AK,ALASKA,2.130451e+07,787.366223,279.696379,9,279.696379,787.366223,alaska,NaN,NaN,NaN,NaN
4,Feature,5,Polygon,"[[[-109.044883728, 36.9986305240002], [-109.04...",AZ,ARIZONA,1.140598e+05,23.752602,28.930993,9,28.930993,23.752602,arizona,NaN,NaN,NaN,NaN


In [15]:
# Defining and using a function to make the headline property.

def add_text(cell): 
    return "This is in district " + cell

In [16]:
merged_df['properties.headline'] = merged_df['properties.District_N'].apply(add_text)

In [17]:
# Setting the article properties.

merged_df['properties.article'] = ''

for i in range(0, len(merged_df)):
    
    district = str(merged_df['district'][i])
    total_orgs = str(merged_df['total_orgs'][i])
    top_judge = str(merged_df['top_judge'][i])        
    top_judge_orgs = str(merged_df['top_judge_orgs'][i])
    if district != 'nan':
        merged_df['properties.article'][i] = district + ' has had ' + str(int(float(total_orgs))) + ' organizations support a federal judge in the last two years. ' + top_judge + ' had the most with ' + str(int(float(top_judge_orgs))) + '.'
    else:
        merged_df['properties.article'][i] = 'No one cares about federal judges in this district! At least, no one has submitted a brief saying so in the last two years.'


    

/var/folders/8w/c7bhb4y56693wcs88hywlmq00000gn/T/ipykernel_21352/477289383.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df['properties.article'][i] = 'No one cares about federal judges in this district! At least, no one has submitted a brief saying so in the last two years.'
/var/folders/8w/c7bhb4y56693wcs88hywlmq00000gn/T/ipykernel_21352/477289383.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df['properties.article'][i] = district + ' has had ' + str(int(float(total_orgs))) + ' organizations support a federal judge in the last two years. ' + top_judge + ' had the most with ' + str(int(floa

In [18]:
# Giving each district a color based on the number of total orgs it has.

# This is less than ideal, I prompted copilot to get the general idea because I'm not super familiar with hex codes.
# The outlier in Washington throws it off, and even after making it a static color the lighter end of the range is too white.

def color(district):
    # get the total orgs for the district
    try:
        total_orgs = merged_df[merged_df.district == district].total_orgs.values[0]

        # These mostly fall between 1 and 50, so we'll use that range.
        # We'll use a linear gradient from light blue to dark blue.
        if total_orgs > 50:
            color = '#%02x%02x%02x' % (0, 0, 255)
        else:
            color = '#%02x%02x%02x' % (int(255 - (total_orgs * 5)*1.1), int(255 - (total_orgs * 5)*1.1), 255)

    except:
        # Generate a gray color if there is no data for the district.
        color = '#%02x%02x%02x' % (128, 128, 128)
    return color

merged_df['properties.color'] = merged_df['district'].apply(color)





In [19]:
# Defining and running functions to make the group dropdown work.

def dis_num(district):
    if district == 'District of Columbia':
        return '12'
    else:
        return district

def dis_name(district):
    if district == 'District of Columbia':
        return district
    else:
        return "District " + district

In [20]:
merged_df['properties.group_id'] = merged_df['properties.District_N'].apply(dis_num)
merged_df['properties.group_name'] = merged_df['properties.District_N'].apply(dis_name)

In [21]:
# All done! Json time.

ok_json = json.loads(merged_df.to_json(orient='records'))

In [22]:
def process_to_geojson(file):
    geo_data = {"type": "FeatureCollection", "features":[]}
    for row in file:
        this_dict = {"type": "Feature", "properties":{}, "geometry": {}}
        for key, value in row.items():
            key_names = key.split('.')
            if key_names[0] == 'geometry':
                this_dict['geometry'][key_names[1]] = value
            if str(key_names[0]) == 'properties':
                this_dict['properties'][key_names[1]] = value
        geo_data['features'].append(this_dict)
    return geo_data

In [23]:
geo_format = process_to_geojson(ok_json)

In [24]:
# variable name
with open('../output/geo-data.js', 'w') as outfile:
    outfile.write("var infoData = ")
# geojson output
with open('../output/geo-data.js', 'a') as outfile:
    json.dump(geo_format, outfile)